### Baseband QPSK System Implementation

The following quadrature phase shift keying (QPSK) system illustrates the core elements of a digital communication system, featuring simplified receiver and transmitter components that are fully synchronized. These core elements are divided into submodules, including bit-to-symbol mapping, upsampling and downsampling of the baseband signal, pulse shaping, and carrier frequency modulation. Each of these modules are summarized in Figure 1 and will be discussed separately below to showcase the functionality of the developed signal processing library and their performance objectives.

<div style="text-align: center;">
    <img src="./images/QPSK/transmitter_receiver_diagram.png" alt="" width="750" />
    <p>QPSK System Transmitter and Receiver Architecture</p>
</div>

#### System Parameters

Before the transmitter architecture can be described, the overall system parameters must be described including the following: sample rate, carrier frequency, and a bit to symbol constellation. A simple test input and header will also be created for use in the later submodules.

In [2]:
import numpy as np

upsample_rate = 8
carrier_frequency = 0.25 * upsample_rate
symbol_clock_offset = 0

# QPSK constellation definition
qpsk_constellation = [
    [complex(np.sqrt(1) + np.sqrt(1) * 1j), 3],
    [complex(np.sqrt(1) - np.sqrt(1) * 1j), 2],
    [complex(-np.sqrt(1) - np.sqrt(1) * 1j), 0],
    [complex(-np.sqrt(1) + np.sqrt(1) * 1j), 1]
]

# Create lists of amplitudes and bits
amplitudes = [i[0] for i in qpsk_constellation]
bits = [i[1] for i in qpsk_constellation]

# Create amplitude to bits and bits to amplitude dictionaries using dictionary comprehensions
amplitude_to_bits = dict(zip(amplitudes, bits))
bits_to_amplitude = dict(zip(bits, amplitudes))

In [3]:
header = [1, 1, 1, 1]
test_input = [1, 0, 1, 0]

# Test input binary
b_k = header + test_input

# Test input symbols
a_k = [bits_to_amplitude[bit] for bit in b_k]

#### Upsample the Baseband Signal

The first step in the transmitter's sequential path is to upsample the test input symbols by the previously defined upsample rate. Upsampling increases the sampling rate of the signal, allowing for finer resolution in the representation of the symbols. This is particularly important in digital communication systems to ensure that the signal can be accurately processed and transmitted.

The upsampling process can be mathematically represented as follows:

$$
x\left(nT_s\right) \rightarrow x_{upsampled}\left(\frac{nT_s}{N}\right)
$$

where: 
- \( N \) is the upsample rate,
- \( T_s \) is the sample duration, 
- \( n \) is the sample index.

During this process, the baseband signal will be split into its real and imaginary components. For demonstration purposes, these components will be visualized as two separate wires. This separation allows for a clearer understanding of how the signal behaves in both the real and imaginary domains, which is crucial for effective modulation and transmission in the later stages of the communication system. By upsampling and visualizing the components, we ensure that the subsequent signal processing steps can be executed with higher fidelity and accuracy.


In [4]:
import sys
sys.path.insert(0, '../KUSignalLib/src') # REMOVE LATER BEFORE SUBMISSION
from KUSignalLib import DSP

a_k_upsampled = DSP.upsample(a_k, upsample_rate, interpolate_flag=False)
a_k_upsampled_real = np.real(a_k_upsampled)
a_k_upsampled_imag = np.imag(a_k_upsampled)

In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(1, 2, figsize=(12, 5))

# Plot original symbols
axs[0].stem(np.imag(a_k[len(header):]))
axs[0].set_title("Original Symbols")
axs[0].set_xlabel("Sample Time [n]")
axs[0].set_ylabel("Amplitude [V]")

# Plot upsampled symbols
axs[1].stem(np.imag(a_k_upsampled[len(header) * upsample_rate:]))
axs[1].set_title("Upsampled Symbols")
axs[1].set_xlabel("Sample Time [n]")
axs[1].set_ylabel("Amplitude [V]")


plt.tight_layout()
plt.show()

#### Pulse Shaping the Baseband Signal

Pulse shaping includes applying some form of windowing, sample and holding, or any other technique with the aim of altering a pulses waveform profile. The square root raised cosine (SRRC) pulse shape is commonly used in digital communication systems for pulse shaping to minimize intersymbol interference (ISI). The SRRC pulse is defined below by its impulse response and is characterized by its roll-off factor $\alpha$.

$$
p\left(nT_s\right)=\frac{1}{\sqrt{N}}\cdot \frac{sin\left(\frac{\pi \left(1-\alpha \right)n}{N}\right)+\frac{4\alpha n}{N}cos\left(\frac{\pi \left(1+\alpha \right)n}{N}\right)}{\frac{\pi n}{N}\cdot \left[1-\left(\frac{4\alpha n}{N}\right)^2\right]}
$$

where: 
- $ N $ is the upsample rate,
- $ T_s $ is the sample duration, 
- $ n $ is the sample index,
- $ \alpha $ is the roll-off factor (0 ≤ $ \alpha $ ≤ 1).

The SRRC pulse shape is applied to the test input signal defined previously and is demonstrated in the block below.

In [12]:
from KUSignalLib import communications

# Length and alpha of SRRC filter
length = 64
alpha = 0.5
pulse_shape = communications.srrc(alpha, upsample_rate, length)

# Applying SRRC to upsampled signal
s_nT_real = np.real(np.roll(DSP.convolve(a_k_upsampled_real, pulse_shape, mode="same"), -1))
s_nT_imag = np.real(np.roll(DSP.convolve(a_k_upsampled_imag, pulse_shape, mode="same"), -1))

In [ ]:
# Plot match filtered signal
plt.figure()
plt.stem(s_nT_imag[len(header)*upsample_rate:])
plt.title("Pulse Shaped Signal")
plt.xlabel("Sample Time [n]")
plt.ylabel("Amplutide [V]")

plt.show()